## Instalar Pacotes

In [1]:
!pip install transformers
!pip install torch

## Carregar dados rotulados por classe

In [2]:
import pandas as pd

In [4]:
tripadvisor_data = pd.read_csv('https://raw.githubusercontent.com/isalbineli/testes/main/BASE_TREINO_LISTA.csv',sep = ',')

df = tripadvisor_data.sample(frac=1).reset_index(drop=True)

df.head()

,review_text,ambiente,comida,geral,preco,servico,seguranca
0,é um local onde as pessoas raramente se reúnem...,0,2,0,2,2,2
1,Lindo.,1,2,1,2,2,2
2,"Hambúrguer pequeno, preço alto, lugar pequeno.",1,1,2,1,2,2
3,É um bar comum mas com uma localização invejáv...,2,2,1,2,2,2
4,"Muito bom, ótimo ambiente.",1,2,2,2,2,2


In [3]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

# Localizar a pasta com os arquivos
import os
os.chdir('/content/drive/MyDrive/MODELOS')

Mounted at /content/drive


In [5]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

## Separar dados de treino e teste

In [6]:
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

## Definir Tokenizador

In [7]:
# Tokenizar os textos
tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased', do_lower_case=False)


tokenizer_config.json:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

## Função para tokenizar os dados

In [8]:
def tokenize_data(data, tokenizer, max_len=200):
    input_ids = []
    attention_masks = []

    for text in data['review_text']:
        encoded_text = tokenizer.encode_plus(
            text,
            max_length=max_len,
            add_special_tokens=True,
            padding='max_length',
            return_attention_mask=True,
            truncation=True,
            return_tensors='pt'
        )

        input_ids.append(encoded_text['input_ids'])
        attention_masks.append(encoded_text['attention_mask'])

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    labels = torch.tensor(data[['ambiente', 'servico', 'comida', 'geral', 'preco', 'seguranca']].values)

    return TensorDataset(input_ids, attention_masks, labels)

In [9]:
train_dataset = tokenize_data(train_data, tokenizer)
test_dataset = tokenize_data(test_data, tokenizer)

## Cria DataLoader

In [10]:
batch_size = 8
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## Modelo BERT para classificação de sequência

In [11]:
model = BertForSequenceClassification.from_pretrained('neuralmind/bert-base-portuguese-cased', num_labels=3)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
from torch import nn
model.classifier = nn.Linear(model.config.hidden_size, 6)

## Função de perda

In [13]:
criterion = torch.nn.CrossEntropyLoss()

## Treinamento do modelo

In [ ]:
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in train_dataloader:
        input_ids, attention_mask, labels = batch

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask)
        loss = criterion(outputs.logits, torch.argmax(labels, dim=1))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_dataloader)
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {avg_loss:.4f}')

## Avaliação do modelo

In [ ]:
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, labels = batch

        outputs = model(input_ids, attention_mask=attention_mask)
        preds = torch.argmax(outputs.logits, dim=1).tolist()
        true_labels = torch.argmax(labels, dim=1).tolist()

        all_preds.extend(preds)
        all_labels.extend(true_labels)

accuracy = accuracy_score(all_labels, all_preds)
classification_rep = classification_report(all_labels, all_preds)

print(f'Accuracy: {accuracy:.4f}')
print('Classification Report:\n', classification_rep)

## Salva Modelo

In [ ]:
# Salvar o modelo
caminho_do_arquivo = '/content/drive/MyDrive/MODELOS/modelo.pth'
torch.save({
    'epoch': num_epochs,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': loss,
}, caminho_do_arquivo)

## Testes

In [ ]:
def analyze_sentiment(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)

    with torch.no_grad():
        outputs = model(**inputs)

    probabilities = torch.nn.functional.softmax(outputs.logits, dim=1).tolist()[0]

    class_mapping = {0: 'ambiente', 1: 'comida', 2: 'geral', 3: 'preco', 4: 'servico', 5: 'seguranca'}
    sentiment_labels = {class_mapping[i]: prob for i, prob in enumerate(probabilities)}

    return sentiment_labels

texto_a_analisar = "Melhor da gastronomia nordestina"
sentimentos = analyze_sentiment(texto_a_analisar, model, tokenizer)

print(f'Sentimentos Previstos: {sentimentos}')

In [ ]:
def analyze_sentiment(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)

    with torch.no_grad():
        outputs = model(**inputs)

    probabilities = torch.nn.functional.softmax(outputs.logits, dim=1).tolist()[0]

    class_mapping = {0: 'ambiente', 1: 'comida', 2: 'geral', 3: 'preco', 4: 'servico', 5: 'seguranca'}
    sentiment_labels = {class_name: int(prob * 3) for class_name, prob in zip(class_mapping.values(), probabilities)}

    return sentiment_labels

texto_a_analisar = "Melhor da gastronomia nordestina"
sentimentos = analyze_sentiment(texto_a_analisar, model, tokenizer)

print(f'Sentimentos Previstos: {sentimentos}')